<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [10]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
response.status_code

200

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [18]:
parameters = {'lat':-33.87,'lon':151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [16]:
json_data

{'message': 'success',
 'timestamp': 1613369126,
 'iss_position': {'latitude': '-35.9292', 'longitude': '-155.9849'}}

Print the response header:

In [19]:
response2 = requests.get("http://api.open-notify.org/iss-pass.json",parameters)

Print the content of the response (the data that the server returned):

In [24]:
response2.text

'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1613369509, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 370, \n      "risetime": 1613374243\n    }, \n    {\n      "duration": 660, \n      "risetime": 1613379869\n    }, \n    {\n      "duration": 581, \n      "risetime": 1613385735\n    }, \n    {\n      "duration": 383, \n      "risetime": 1613391722\n    }, \n    {\n      "duration": 400, \n      "risetime": 1613397630\n    }\n  ]\n}\n'

Note that this is a Python byte string:

In [25]:
response2.text

'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1613369509, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 370, \n      "risetime": 1613374243\n    }, \n    {\n      "duration": 660, \n      "risetime": 1613379869\n    }, \n    {\n      "duration": 581, \n      "risetime": 1613385735\n    }, \n    {\n      "duration": 383, \n      "risetime": 1613391722\n    }, \n    {\n      "duration": 400, \n      "risetime": 1613397630\n    }\n  ]\n}\n'

Print just the "content-type" value from the header:

In [30]:
response2.headers['Content-Type']

'application/json'

JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [34]:
print(response2.json())

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1613369509, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 370, 'risetime': 1613374243}, {'duration': 660, 'risetime': 1613379869}, {'duration': 581, 'risetime': 1613385735}, {'duration': 383, 'risetime': 1613391722}, {'duration': 400, 'risetime': 1613397630}]}


What kind of object did this give us?

In [35]:
type(response2.json())

dict

In [41]:
response2.json().keys()

dict_keys(['message', 'request', 'response'])

Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [42]:
response_dict = response2.json()['response']

In [45]:
response_dict

[{'duration': 370, 'risetime': 1613374243},
 {'duration': 660, 'risetime': 1613379869},
 {'duration': 581, 'risetime': 1613385735},
 {'duration': 383, 'risetime': 1613391722},
 {'duration': 400, 'risetime': 1613397630}]

Now extract the `risetime` strings into an array called `srisetimes`:

In [49]:
srisetimes = []
for i in range(len(response_dict)):
    srisetimes.append(response_dict[i]['risetime'])
srisetimes

[1613374243, 1613379869, 1613385735, 1613391722, 1613397630]

```
datetime.fromtimestamp(str)
```

It is Unix Timestamp. Convert these to an array of Python `datetime` values called `risetimes`:

In [64]:
datetime.fromtimestamp(srisetimes[0])
list_of_datetimes = map(datetime.fromtimestamp, srisetimes)
risetimes = list(list_of_datetimes)
risetimes

[datetime.datetime(2021, 2, 15, 15, 30, 43),
 datetime.datetime(2021, 2, 15, 17, 4, 29),
 datetime.datetime(2021, 2, 15, 18, 42, 15),
 datetime.datetime(2021, 2, 15, 20, 22, 2),
 datetime.datetime(2021, 2, 15, 22, 0, 30)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [74]:
parsed_datetimes = []
for element in risetimes:
     parsed_datetimes.append(element.strftime('%d/%m/%y %I:%M'))
parsed_datetimes

['15/02/21 03:30',
 '15/02/21 05:04',
 '15/02/21 06:42',
 '15/02/21 08:22',
 '15/02/21 10:00']

Here is an endpoint that tells us who is onboard:

In [75]:
response3 = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [77]:
astronaut_data = json.loads(response3.text)
astronaut_data.keys()

dict_keys(['message', 'number', 'people'])

In [87]:
astronaut_data['people']

name_list = []
for person in astronaut_data['people']:
    name_list.append(person['name'])

print('Number of people is {n_people} and their names are {names}'.format(n_people=len(astronaut_data['people']),names=name_list))

Number of people is 7 and their names are ['Sergey Ryzhikov', 'Kate Rubins', 'Sergey Kud-Sverchkov', 'Mike Hopkins', 'Victor Glover', 'Shannon Walker', 'Soichi Noguchi']


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [115]:
#ANSWER:
def handleResponse(response):
    '''
    Returns Boolean Value, Status Code, 
    '''
    if response.status_code == 200:
        return False, response.status_code
    else:
        return True, response.status_code
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [117]:
response = requests.get("http://api.open-notify.org/astros.json")
a,b = handleResponse(response)
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
x,y = handleResponse(response)
a,b,x,y

API call failed. Resolve issue before continuing!


(False, 200, True, 400)

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



